### 1. 데이터 준비

In [8]:
!apt install unzip
!unzip dog.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Archive:  dog.zip
  inflating: train/bo/bo_1.jpg       
  inflating: train/bo/bo_10.jpg      
  inflating: train/bo/bo_11.jpg      
  inflating: train/bo/bo_12.jpg      
 extracting: train/bo/bo_13.jpg      
  inflating: train/bo/bo_14.jpg      
  inflating: train/bo/bo_15.jpg      
  inflating: train/bo/bo_16.jpg      
  inflating: train/bo/bo_17.jpg      
  inflating: train/bo/bo_18.jpg      
  inflating: train/bo/bo_19.jpg      
  inflating: train/bo/bo_2.jpg       
  inflating: train/bo/bo_3.jpg       
  inflating: train/bo/bo_4.jpg       
  inflating: train/bo/bo_5.jpg       
  inflating: train/bo/bo_6.jpg       
  inflating: train/bo/bo_7.jpg       
  inflating: train/bo/bo_8.jpg       
  inflating: train/bo/bo_9.jpg       
  inflating: train/not_bo/1.jpg      
  infl

In [ ]:
import numpy as np  ## NumPy 라이브러리를 가져와 배열 및 행렬 연산을 수행
import cv2  ## OpenCV 라이브러리를 가져와 이미지 처리 및 컴퓨터 비전 작업을 수행
import glob ## glob 모듈을 가져와 파일경로 패턴 매칭을 수행
from PIL import Image  ## PIL(Python Imaging Library)에서 Image 모듈을 가져와 이미지 파일을 처리

import torch  ## PyTorch 라이브러리를 가져와 딥러닝 모델을 구축하고 학습
import torch.nn as nn  #PyTorch의 신경망 모듈을 가져와 신경망 레이어를 정의
from torch.utils.data import Dataset, DataLoader  ## PyTorch 데이터 로더를 가져와 데이터셋을 배치로 로드

from torchvision import transforms as T  ## Torchvision의 변환 (transforms) 모듈을 가져와 이미지에 대한 전처리 및 증강을 수행
import torchvision.models as models ## Torchvision의 시전 훈련된 모델을 가져와 전이 학습에 사용

In [ ]:
## 이미지 전처리를 위한 변환(transform) 파이프라인을 정의
## T.Compose를 사용 -> 여러 변환을 순차적으로 적용
## 1. T.Resize(256): 이미지를 256X256 크기로 리사이즈
## 2. T.RandomCrop(244) : 리 사이즈된 이미지에서 244X244 크기의 랜덤한 영역을 잘라냄
## 3. T.ToTensor() : PIL 이미지나 NumPy 배열을 PyTorch 텐서로 변환
transform = T.Compose([T.Resize(256), T.RandomCrop(244), T.ToTensor()])

class Dataset(Dataset):  ## 사용자 정의 데이터셋 클래스를 정의 , PyTorch의 Dataset 클래스를 상속
    def __init__(self, path='train'): ## 초기화 메서드 , 데이터 셋의 경로르 받아 이미지 파일 경로와 레이블을 설정
        super(Dataset, self).__init__()  ## 부모 클래스의 초기화 메서드를 호출

        bo_path = glob.glob(path+'/bo/*.jpg')  ## 'bo' 폴더 내의 모든 .jpg 파일 경로를 가져옴
        notbo_path = glob.glob(path+'/not_bo/*.jpg')  ## 'not bo' 폴더 내의 모든 .jpg 파일 경로를 가져옴
        self.img_path = bo_path + notbo_path  ## 'bo'와 'not bo'의 이미지 경로를 합쳐 전체 이미지 경로 리스트를 만듬
        self.label_list = [1]*len(bo_path) + [0]*len(notbo_path)  ## 'bo' 이미지에는 레이블 1, 'not bo' 이미지에는 레이블 0을 할당 -> 레이블 리스트를 만듬

    def __getitem__(self, index):  ## 데이터셋의 특정 인덱스에 해당하는 샘플을 가져오는 메서드
        img = cv2.imread(self.img_path[index])  ## 지정된 인덱스의 이미지 파일을 OpenCV를 사용하여 읽음
        img_pil = Image.fromarray(img)  ## OpenCV로 읽은 이미지를 PIL 이미지로 전처리하고 텐서로 변환
        self.img_tensor = transform(img_pil)  ## 정의된 transform을 사용하여 PIL 이미지를 전처리 하고 텐서로 변환
        self.label_tensor = torch.tensor(self.label_list[index])  ## 해당 이미지의 레이블을 PyTorch 텐서로 변환
        return self.img_tensor.to("cuda:0"), self.label_tensor.to("cuda:0")  ## 이미지 텐서와 레이블 텐서를 GPU로 이동하여 반환
## 데이터셋의 전체 길이 (샘플 수)를 반환하는 메서드
    def __len__(self):
        return len(self.img_path)

In [ ]:
## 'train' 폴더를 사용해 학습용 데이터셋을 생성
training_dataset = Dataset('train')
## 'vali' 폴더를 사용해 검증용 데이터셋을 생성
validation_dataset = Dataset('valid')

In [ ]:
## 학습용 데이터 로더를 생성
## - dataset : 학습용 데이터셋을 사용
## - batch_size : 한 번에 처리할 데이터 샘플의 수를 8로 설정
## - shuffle : 검증 데이터는 섞지 않고 순서대로 사용 , 이는 성능 평가의 일관성을 유지하기 위함
training_loader = DataLoader(dataset=training_dataset, batch_size=8, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=8, shuffle=False)

### 2. 뉴럴네트워크 모델링

In [ ]:
## VGG16 모델을 불러옴 , 사전 훈련된 가중치를 사용하지 않고 -> 무작위로 초기화된 모델을 생성
#vgg16 = models.vgg16()
## VGG16 모델을 불러오고 , ImageNet 데이터셋으로 사전 훈련된 가중치를 사용
vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 76.2MB/s]


In [ ]:
## VGG16 모델의 모든 파라미터에 대해 반복
for param in vgg16.parameters():
## 각 파라미터의 REquires_grad 속성을 False로 설정 , 해당 ㅍ파라미터가 학습되지 않도록 함
    param.requires_grad = False

In [ ]:
## VGG16 모델의 classifier 레이어 중 첫 번째 레이어 ( fully connected layer )의 입력 특징 수를 가져옴
## 이는 새로운 classifier를 설계할 떄 입력 크기를 맞추기 위해 필요
num_features = vgg16.classifier[0].in_features
## VGG16의 classifier 부분을 새로운 Sequential 모듈로 대체
## 이 Sequential 모듈은 여러 레이어를 순차적으로 쌓아 구성 , 다음과 같은 레이어로 이루어짐
## 1. nn.Linear(num_features, 256) : 입력 특징 수에서 256개의 뉴런으로 연결된 fully connected layer
## 2. NN.ReLU() : 활성화 함수로 ReLU를 사용해 비선형성을 추가
## 3. nn.Linear(256, 2) : 256개의 뉴런에서 2개의 출력 클래스로 연결된 fully connected layer
vgg16.classifier = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Linear(256, 2)
)

In [ ]:
## VGG16 모델을 GPU로 이동 , ' cuda:0'은 첫 번쨰 GPU를 의미 , 이를 통해 GPU가속을 사용 -> 더 빠른 계산을 가능하게 함
vgg16 = vgg16.to('cuda:0')

### 3. 손실함수와 최적화기법 정의

In [ ]:
## 손실 함수로 CrossEntropyLoss를 사용 , 이 함수는 다중 클래스 분류 문제에서 자주 사용
## 모델의 예측과 실제 레이블 간의 차이를 측정하여 모델이 학습할 수 있도록 함
loss_function = nn.CrossEntropyLoss()
## 최적화 도구로 Adam을 사용 , Adam은 학습률을 자동으로 조정하는 알고리즘
## VGG16 모델의 파라미터를 업데이터하는데 사용 , 학습률(lr)은 0.0001로 설정
optimizer = torch.optim.Adam(vgg16.parameters(), lr=0.0001)

### 3. 뉴럴네트워크 훈련

In [ ]:
for epoch in range(10):  ## 10d 에포크 동안 반복 (각 에포크는 데이터셋 전체를 한 번 학습)
    loss_val = 0  ## 각 에포크 시작 시 손실 값을 0으로 초기화 , 에포크 동안 누적할 예정
    for itr, data in enumerate(training_loader):  ##학습 데이터 로더에서 배치 단위로 데이터 반복
        optimizer.zero_grad()  ## 이전 그래디언트를 0으로 초기화 , 새로운 그래디언트 계산 준비
        inputs, labels = data  ## 배치 데이터를 입력과 레이블로 분리
        pred = vgg16(inputs)  ## VGG16 모델에 입력을 전달하여 예측값 계산
        loss = loss_function(pred, labels)  ## 예측값과 실제 레이블 간의 손실 계산

        loss.backward()  ## 손실에 대해 연적파 수행 , 그래디언트 계산

        optimizer.step()  ## 계산된 그래디언트를 기반으로 모델 파라미터 업데이트

        loss_val += loss.item()  ## 현재 배치의 손실 값을 누적 ( Python 숫자로 변환 )
    print("Epoch: ", epoch+1, " , Loss: ", loss_val)  ## 에포크 번호 ( 1부터 시작 )와 총 손실 출력

Epoch:  1  , Loss:  8.621671382337809
Epoch:  2  , Loss:  1.7960744048468769
Epoch:  3  , Loss:  0.3592872351873666
Epoch:  4  , Loss:  0.273036266095005
Epoch:  5  , Loss:  0.09288019145606086
Epoch:  6  , Loss:  0.07390306067827623
Epoch:  7  , Loss:  0.05486678182205651
Epoch:  8  , Loss:  0.03361212946037995
Epoch:  9  , Loss:  0.026950100000249222
Epoch:  10  , Loss:  0.023187731254438404


### 5. 성능평가

In [ ]:
pred_list = []  ## 예측된 카테고리를 저장할 리스트 초기화
label_list = []  ## 실제 레이블을 저장할 리스트 초기화

for itr, data in enumerate(validation_loader):  ## 검증 데이터 로더에서 배치 단위로 데이터 반복
    inputs, labels = data  ## 배치 데이터를 입력과 레이블로 분리

    pred = vgg16(inputs)  ## VGG16 모델에 입력을 전달하여 예측값 계산
    pred_category = torch.argmax(pred, dim=1)  ## 예측값에서 가장 높은 확률의 카테고리 인덱스 추출

    pred_list = pred_list + list(pred_category.cpu())  ## 예측된 카테고리를 CPU로 이동 후 리스트에 추가
    label_list = label_list + list(labels.cpu())  ## 실제 레이블을 CPU로 이동 후 리스트에 추가

In [ ]:
accu = np.mean( np.array(pred_list) == np.array(label_list) )  ## 예측값과 실제 레이블을 비교하여 정확도 계산 ( 평균 )
print("Validation accuracy: ", accu*100)  ## 검증 정확도를 백분율로 변환하여 출력

Validation accuracy:  96.66666666666667
